# Simple Walk 1D

## Create a simple environment where you can walk around

input is the direction up, down, left or right
try to reach a goal
terminate if you reach goal or cross border
reward is -distance for goal and -1 for step

base environment on https://github.com/nicknochnack/ReinforcementLearningCourse/blob/main/Project%203%20-%20Custom%20Environment.ipynb

## 1. Import Dependencies

In [2]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [57]:
class SimpleWalk1D(Env):
    def __init__(self):
        # go left or right
        self.action_space = Discrete(2) # 0: left, 1: right
        
        self.observation_space = MultiDiscrete([10, 10])
        
        self.state = (random.randint(0,9), random.randint(0,9))
        
        # set max steps
        self.steps = 10
        
        
    def step(self, action):
        self.steps -= 1
        
        # calculate correct action
        action2 = action * 2 - 1
        
        # update state
        self.state = (self.state[0] + action2, self.state[1])
        
        if self.state[0] == -1 or self.state[0] == 10:
            # went over bounds
            done = True
            reward = -10
        elif self.state[0] == self.state[1]:
            # reached goal
            done = True
            reward = 10
        elif self.steps == 0:
            # ran out of steps
            done = True
            reward = -10
        else:
            done = False
            reward = -1

        return self.state, reward, done, {}

    def render(self):
        pass
    
    def reset(self):
        self.state = (random.randint(0,9), random.randint(0,9))
        self.steps = 10
        return self.state
        
        
        
        
        
        

In [74]:
env = SimpleWalk1D()
type(env.reset())

tuple

In [75]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-10
Episode:2 Score:-19
Episode:3 Score:10
Episode:4 Score:-19
Episode:5 Score:-16


In [76]:
log_path = os.path.join('Training', 'Logs')

In [77]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [78]:
model.learn(total_timesteps=400000)

Logging to Training\Logs\PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 5.76     |
|    ep_rew_mean     | -8.16    |
| time/              |          |
|    fps             | 275      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.02        |
|    ep_rew_mean          | -6.22       |
| time/                   |             |
|    fps                  | 306         |
|    iterations           | 2           |
|    time_elapsed         | 13          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015370306 |
|    clip_fraction        | 0.159       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.00325     |

KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir log_path

In [79]:
model.save('PPO')

In [81]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

C:\Users\Luttk\anaconda3\envs\rl-test\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(6.5, 1.9621416870348585)